In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp
import kalepy as kale

import holodeck as holo
import holodeck.anisotropy as anis
from holodeck import detstats, plot, utils
from holodeck.constants import YR, MSOL, GYR

# Set Up

In [ ]:
dur, cad = 16.03*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
fobs_orb_cents = fobs_gw_cents/2.0
fobs_orb_edges = fobs_gw_edges/2.0

# sam = holo.sam.Semi_Analytic_Model()
sam_20 = holo.sam.Semi_Analytic_Model(shape=20)  # faster version
hard_FT20 = holo.hardening.Fixed_Time_2PL_SAM(sam_20, 3*GYR)

def setup_calculate_all(sam, hard):
    redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)
    edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
    if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
        hard_name = 'Fixed Time'
    elif isinstance(hard, holo.hardening.Hard_GW):
        hard_name = 'GW Only'


    hs = holo.gravwaves.strain_amp_from_bin_edges_redz(edges, redz_final)
    cynum = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)
    C0_cynum, Cl_cynum = anis.Cl_analytic_from_num(fobs_orb_edges, number=cynum, hs=hs)
    C0_cyreals, Cl_cyreals = anis.Cl_analytic_from_num(fobs_orb_edges, cynum, hs, realize=20)

    utnum = utils._integrate_grid_differential_number(edges, diff_num, freq=False)
    utnum = utnum * np.diff(np.log(fobs_gw_edges))
    C0_utnum, Cl_utnum = anis.Cl_analytic_from_num(fobs_orb_edges, number=utnum, hs=hs)
    C0_utreals, Cl_utreals = anis.Cl_analytic_from_num(fobs_orb_edges, utnum, hs, realize=20)

    C0_dnum, Cl_dnum = anis.Cl_analytic_from_dnum(edges, diff_num)
    C0_dnum_reals, Cl_dnum_reals = anis.Cl_analytic_from_dnum(edges, diff_num, realize=10)
    C0_redz, Cl_redz = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final)
    C0_redz_reals, Cl_redz_reals = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final, realize=10)

    C0_floor, Cl_floor = anis.Cl_analytic_from_num(fobs_orb_edges, number=cynum, hs=hs, realize=False, floor=True)
    flnum = np.floor(cynum)
    C0_flr2, Cl_flr2 = anis.Cl_analytic_from_num(fobs_orb_edges, flnum, hs, realize=False, floor=False)
    
    vals = {
        'sam':sam, 'hard': hard, 'hard_name':hard_name, 'hs':hs,
        'edges':edges, 'redz_final':redz_final, 'diff_num':diff_num,
        'cynum':cynum, 'C0_cynum':C0_cynum, 'Cl_cynum':Cl_cynum, 'Cl_cyreals':Cl_cyreals, 'C0_cyreals':C0_cyreals,
        'utnum':utnum, 'C0_utnum':C0_utnum, 'Cl_utnum':Cl_utnum, 'Cl_utreals':Cl_utreals, 'C0_utreals':C0_utreals,
        'C0_dnum':C0_dnum, 'Cl_dnum':Cl_dnum, 'C0_dnum_reals':C0_dnum_reals, 'Cl_dnum_reals':Cl_dnum_reals,
        'C0_redz':C0_redz, 'Cl_redz':Cl_redz, 'C0_redz_reals':C0_redz_reals, 'Cl_redz_reals':Cl_redz_reals,
        'flnum':flnum, 'C0_floor':C0_floor, 'Cl_floor':Cl_floor, 'C0_flr2':C0_flr2, 'Cl_flr2':Cl_flr2,
    }



    return vals    

vals_FT20 = setup_calculate_all(sam_20, hard_FT20)

# Calculate Anisotropy

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{f}{4\pi \Delta f}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^2 (f,\vec{\theta})   \bigg)^2 
+ \big( \frac{f}{4 \pi \Delta f}\big)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^4 (f, \vec{\theta})
$$



* dens = d^3 n / [dlog10M dq dz] in units of [Mpc^-3] 
* dnum = d^4N / dlog10M dq dz dlnf
* number = dN /dlnf

 ### Plot Cl_analytic_from_num()

In [ ]:
def plot_Cl_analytic_from_num(vals): # uses nb variables
    Cl_cynum = vals['Cl_cynum']
    C0_cynum = vals['C0_cynum']
    Cl_cyreals = vals['Cl_cyreals']
    C0_cyreals = vals['C0_cyreals']

    Cl_utnum = vals['Cl_utnum']
    C0_utnum = vals['C0_utnum']
    Cl_utreals = vals['Cl_utreals']
    C0_utreals = vals['C0_utreals']

    Cl_floor = vals['Cl_floor']
    C0_floor = vals['C0_floor']
    Cl_flr2 = vals['Cl_flr2']
    C0_flr2 = vals['C0_flr2']

    fig1 = anis.plot_ClC0_versions(fobs_gw_cents)
    ax = fig1.axes[0]
    ax.set_title(vals['hard_name']+', '+str(vals['sam'].shape))

    anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', alpha=0.5, lw=4)
    anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                    show_reals=True, show_median=True, show_ci=True)

    anis.draw_analytic(ax, Cl_utnum, C0_utnum, fobs_gw_cents, color='tab:red', label='utils number', alpha=0.5)
    anis.draw_reals(ax, Cl_utreals, C0_utreals, fobs_gw_cents, color='tab:red', label=None,
                                    show_reals=True, show_median=True, show_ci=True)



    anis.draw_analytic(ax, Cl_floor, C0_floor, fobs_gw_cents, color='tab:blue', label='floor number', alpha=0.5)

    anis.draw_analytic(ax, Cl_flr2, C0_flr2, fobs_gw_cents, color='tab:green', label='flr2 number', alpha=0.5, lw=5)

    fig1.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=4)

    fig1.tight_layout()
    return fig1

fig = plot_Cl_analytic_from_num(vals_FT20)

### Plot number

In [ ]:
def plot_num_sums(vals):
    cynum = vals['cynum']
    flnum = vals['flnum']

    fig, (ax1, ax2) = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$\sum_{M,q,z} N$', nrows=2, sharex=True)

    xx = fobs_gw_cents
    y1 = np.sum(cynum, axis=(0,1,2))
    y2 = np.sum(flnum, axis=(0,1,2))
    labels = np.array(['number', 'num rounded down'])

    for ii,yy in enumerate([y1, y2]):
        ax1.plot(xx, yy, label=labels[ii], alpha=0.5)
    ax1.legend()
    ax1.set_title(vals['hard_name']+', '+str(vals['sam'].shape))


    ax2.plot(xx, y2/y1, label='rounded / not rounded')
    ax2.legend()

    fig.tight_layout()
    return fig

fig = plot_num_sums(vals_FT20)

This confirms that using number function is same, regardless which number we use

### Plot hs^2 and hs^4

In [ ]:
def plot_hs2_and_hs4(vals):
    hs = vals['hs']
    
    xx = fobs_gw_cents
    fig, axs = plot.figax(ncols=2,figsize=(8.5,3),
        xlabel=plot.LABEL_GW_FREQUENCY_HZ, sharex=True)
    
    draw_hs2_hs4(axs, xx, hs)
    axs[0].set_title(vals['hard_name']+', '+str(vals['sam'].shape))

    fig.tight_layout()
    return fig
    

def draw_hs2_hs4(axs, xx, hs):
    y1 = np.sum(hs**2, axis=(0,1,2))
    y2 = np.sum(hs**4, axis=(0,1,2))
    labels = np.array(['$\sum_{M,q,z} h_s^2$', '$\sum_{M,q,z} h_s^4$'])


    for ii, yy in enumerate([y1, y2]):
        axs[ii].scatter(xx, yy)
        axs[ii].set_ylabel(labels[ii])

def plot_numh2_and_numh4(vals):
    hs = vals['hs']
    cynum = vals['cynum']
    flnum = vals['flnum']
    
    xx = fobs_gw_cents
    fig, axs = plot.figax(ncols=2,figsize=(8.5,3),
        xlabel=plot.LABEL_GW_FREQUENCY_HZ, sharex=True)

    draw_numh2_numh4(axs, xx, hs, cynum, flnum)
    axs[0].set_title(vals['hard_name']+', '+str(vals['sam'].shape))
    fig.tight_layout()
    return fig

def draw_numh2_numh4(axs, xx, hs, cynum, flnum):
    cy1 = np.sum(hs**2*cynum, axis=(0,1,2))
    cy2 = np.sum(hs**4*cynum, axis=(0,1,2))
    yy_cy = np.array([cy1, cy2])
    fl1 = np.sum(hs**2*flnum, axis=(0,1,2))
    fl2 = np.sum(hs**4*flnum, axis=(0,1,2))
    yy_fl = np.array([fl1, fl2])
    cylabel = 'num'
    fllabel = 'rounded'
    ylabels = np.array(['$\sum_{M,q,z} N h_s^2$', '$\sum_{M,q,z} N h_s^4$'])


    for ii, ax in enumerate(axs):
        ax.scatter(xx, yy_cy[ii], label=cylabel)
        ax.scatter(xx, yy_fl[ii], label=fllabel)
        ax.set_ylabel(ylabels[ii])
        ax.legend()

def plot_h2_h4_numh2_numh4(vals):
    hs = vals['hs']
    cynum = vals['cynum']
    flnum = vals['flnum']
    
    xx = fobs_gw_cents
    fig, axs = plot.figax(ncols=2,nrows=2, figsize=(8.5,4.5),
        sharex=True) 
    for ax in axs[1,:]:
        ax.set_xlabel(plot.LABEL_GW_FREQUENCY_HZ)
    draw_hs2_hs4(axs[0,:], xx, hs)
    draw_numh2_numh4(axs[1,:], xx, hs, cynum, flnum)
    axs[0,0].set_title(vals['hard_name']+', '+str(vals['sam'].shape))
    
    fig.tight_layout()
    return fig



fig = plot_hs2_and_hs4(vals_FT20)
fig = plot_numh2_and_numh4(vals_FT20)
fig = plot_h2_h4_numh2_numh4(vals_FT20)


#### FT, Shape 20

In [ ]:
vals = vals_FT20
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)


#### FT, Shape 40

In [ ]:
sam_40 = holo.sam.Semi_Analytic_Model(shape=40)
hard_FT40 = holo.hardening.Fixed_Time_2PL_SAM(sam_40, 3*GYR)
vals_FT40 = setup_calculate_all(sam_40, hard_FT40)

In [ ]:
vals = vals_FT40
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)

#### FT, Shape full

In [ ]:
sam_full = holo.sam.Semi_Analytic_Model()
hard_FTfull = holo.hardening.Fixed_Time_2PL_SAM(sam_full, 3*GYR)
vals_FTfull = setup_calculate_all(sam_full, hard_FTfull)

In [ ]:
vals = vals_FTfull
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)

#### GW, Shape 20

In [ ]:
hard_GW = holo.hardening.Hard_GW()
vals_GW20 = setup_calculate_all(sam_20, hard_GW)

In [ ]:
vals = vals_GW20
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)

#### GW, Shape 40

In [ ]:
vals_GW40 = setup_calculate_all(sam_40, hard_GW)

In [ ]:
vals = vals_GW40
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)

#### GW, Shape full

In [ ]:
vals_GWfull = setup_calculate_all(sam_full, hard_GW)

In [ ]:
vals = vals_GWfull
fig = plot_Cl_analytic_from_num(vals)
fig = plot_num_sums(vals)
# fig = plot_hs2_and_hs4(vals)
# fig = plot_numh2_and_numh4(vals)
fig = plot_h2_h4_numh2_numh4(vals)

## Cl_analytic_from_dnum

note that Cl_best does not use the same model as the mockups for Sato-Polito method here!

In [ ]:

C0_dnum = vals_FT20['C0_dnum']
Cl_dnum = vals_FT20['Cl_dnum']
C0_dnum_reals = vals_FT20['C0_dnum_reals']
Cl_dnum_reals = vals_FT20['Cl_dnum_reals']
C0_redz = vals_FT20[ 'C0_redz']
Cl_redz = vals_FT20['Cl_redz']
C0_redz_reals = vals_FT20['C0_redz_reals'] 
Cl_redz_reals = vals_FT20['Cl_redz_reals']

fig = anis.plot_ClC0_versions(fobs_gw_cents)
ax = fig.axes[0]

anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', alpha=0.5, lw=4)
anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                show_reals=True, show_median=True, show_ci=True)

anis.draw_analytic(ax, Cl_dnum, C0_dnum, fobs_gw_cents, label='dnum, z_init', color='deeppink')
anis.draw_reals(ax, Cl_dnum_reals, C0_dnum_reals, fobs_gw_cents, label=None, color='deeppink')

anis.draw_analytic(ax, Cl_redz, C0_redz, fobs_gw_cents, label='dnum, z_final', color='indigo')
anis.draw_reals(ax, Cl_redz_reals, C0_redz_reals, fobs_gw_cents, label=None, color='indigo')

fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=4)
ax.set_title('Shape=%s, %s' % (str(sam.shape), str(hard_name)), fontsize=14)

fig.tight_layout()


# Compare Models

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=10)
hard = holo.hardening.Hard_GW()
print(sam._density)
fobs_orb_cents = fobs_gw_cents/2.0
fobs_orb_edges = fobs_gw_edges/2.0
redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)
print(sam._density.shape)

In [ ]:
def compare_all_analytic_anis(sam, hard):

    redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
        fobs_orb_cents, sam, hard, holo.cosmo)
    edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
    if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
        hard_name = 'Fixed Time'
    elif isinstance(hard, holo.hardening.Hard_GW):
        hard_name = 'GW Only'

    # analytic from cython number
    print('calculating analytic from cython number')
    hs = holo.gravwaves.strain_amp_from_bin_edges_redz(edges, redz_final)
    cynum = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)
    C0_cynum, Cl_cynum = anis.Cl_analytic_from_num(fobs_orb_edges, number=cynum, hs=hs)
    C0_cyreals, Cl_cyreals = anis.Cl_analytic_from_num(fobs_orb_edges, cynum, hs, realize=20)

    # # anayltic from utils number
    # utnum = utils._integrate_grid_differential_number(edges, diff_num, freq=False)
    # utnum = utnum * np.diff(np.log(fobs_gw_edges))
    # C0_utnum, Cl_utnum = anis.Cl_analytic_from_num(fobs_orb_edges, number=utnum, hs=hs)
    # C0_utreals, Cl_utreals = anis.Cl_analytic_from_num(fobs_orb_edges, utnum, hs, realize=20)

    # anayltic from floor (rounded) number
    flnum = np.floor(cynum)
    C0_flnum, Cl_flnum = anis.Cl_analytic_from_num(fobs_orb_edges, number=flnum, hs=hs)

    # analytic from dnum, zinit
    print('calculating analytic from dnum, initial redshift')
    C0_init, Cl_init = anis.Cl_analytic_from_dnum(edges, diff_num)
    C0_inreals, Cl_inreals = anis.Cl_analytic_from_dnum(edges, diff_num, realize=10)

    # analytic from dnum, zfinal
    print('calculating analytic from dnum, final redshift')
    C0_redz, Cl_redz = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final)
    C0_rzreals, Cl_rzreals = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final, realize=10)

     # plot everything
    print('plotting')
    fig = anis.plot_ClC0_versions(fobs_gw_cents)
    ax = fig.axes[0]

    anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', lw=4)
    anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                    show_reals=True, show_median=True, show_ci=True)

    anis.draw_analytic(ax, Cl_flnum, C0_flnum, fobs_gw_cents, color='tab:red', label='rounded number', lw=4)
    # anis.draw_reals(ax, Cl_utreals, C0_utreals, fobs_gw_cents, color='tab:red', label=None,
    #                 show_reals=True, show_median=True, show_ci=True)

    anis.draw_analytic(ax, Cl_init, C0_init, fobs_gw_cents, label='dnum, z_init', color='deeppink', lw=4)
    anis.draw_reals(ax, Cl_inreals, C0_inreals, fobs_gw_cents, label=None, color='deeppink')

    anis.draw_analytic(ax, Cl_redz, C0_redz, fobs_gw_cents, label='dnum, z_final', color='indigo', lw=4)
    anis.draw_reals(ax, Cl_rzreals, C0_rzreals, fobs_gw_cents, label=None, color='indigo')

    fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=4)
    ax.set_title('%s, %s' % ( str(hard_name), str(sam.shape)), fontsize=12)

    fig.tight_layout()


    vals = {
        'sam':sam, 'hard':hard, 'edges':edges, 'redz_final':redz_final, 'diff_num':diff_num, 'hs':hs,
        'cynum':cynum, 'C0_cynum':C0_cynum, 'Cl_cynum':Cl_cynum, 'C0_cyreals':C0_cyreals, 'Cl_cyreals':Cl_cyreals, 
        'flnum':flnum, 'C0_flnum':C0_flnum, 'Cl_flnum':Cl_flnum,
        'C0_init':C0_init, 'Cl_init':Cl_init, 'C0_inreals':C0_inreals, 'Cl_inreals':Cl_inreals,
        'C0_redz':C0_redz, 'Cl_redz':Cl_redz, 'C0_rzreals':C0_rzreals, 'Cl_rzreals':Cl_rzreals,

    }
    return fig, vals

#### FT, Shape 20

In [ ]:
fig, vals = compare_all_analytic_anis(sam_20, hard_FT20)

#### FT, Shape 40

In [ ]:
fig, vals = compare_all_analytic_anis(sam_40, hard_FT40)

#### FT, Shape full

In [ ]:
fig, vals = compare_all_analytic_anis(sam_full, hard_FTfull)

#### GW, Shape 20

In [ ]:
fig, vals = compare_all_analytic_anis(sam_20, hard_GW)

#### GW, Shape 40

In [ ]:
fig, vals = compare_all_analytic_anis(sam_40, hard_GW)

#### GW, Shape full

In [ ]:
fig, vals = compare_all_analytic_anis(sam_full, hard_GW)

# Remaining Questions

Why is np.random(number) * hs^2 from  Cl_analytic_from_num different from np.random(integral(dnum)) * hs^2 in Cl_analytic_from_dnum?

Because, in Cl_analytic_from_num we take np.random after multiplying by d/dlnf, whereas in Cl_analytic_from_dnum, we are doing an extra *dif(log(fobs)) that we shouldn't be doing!!